In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE157738"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE157738"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE157738.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE157738.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE157738.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE157738_family.soft.gz', 'GSE157738_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Melanoma/GSE157738/GSE157738_family.soft.gz
Matrix file: ../../input/GEO/Melanoma/GSE157738/GSE157738_series_matrix.txt.gz


Background Information:
!Series_title	"Monocyte-derived DC Expression Data from Advanced Staged Melanoma Patients"
!Series_summary	"The metastatic form of Melanoma has a reported ten-year survival rate of approximately 15%. Clinical trials have shown modest success in a subset of patients. Particularly, combinational therapy using checkpoint blockade has shown the most success, but many patients do not respond. The patients that do respond to treatments often have a pre-existing antitumor immunity."
!Series_summary	"To generate an optimal anti-tumor immune response, we have previously created a dendritic cell (DC) based adenovirus vaccine targeted against three common melanoma associated antigens: Tyrosinase, MART-1, and MAGE-A6 (TMM2). The vaccine was used in a Phase 1 clinical trial (NCT01622933) , where 35 patients were enrolled. Immature DC (iDC) were generated from patient monocytes (GM-CSF + IL-4), matured (mDC) using IFNG + LPS, and transduced with the adenovirus vaccine (AdVTMM

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
import gzip
from typing import Optional, Callable, Dict, Any

# 1. Determine if gene expression data is available
# Based on the series title and summary, this dataset contains gene expression data from Affymetrix Human Gene 2.0 ST Array
is_gene_available = True

# 2. Identify rows for trait (melanoma), age, and gender in the sample characteristics dictionary
# Looking at the sample characteristics dictionary:
# Row 4 contains 'clinical outcome' which can be used as our trait (response to treatment)
trait_row = 4
# There's no age information available in the sample characteristics
age_row = None
# There's no gender information available in the sample characteristics
gender_row = None

# 2.2 Data type conversion functions
def convert_trait(value):
    """Convert clinical outcome to binary for melanoma response."""
    if not value or ':' not in value:
        return None
    
    # Extract the value after the colon and strip whitespace
    clinical_outcome = value.split(':', 1)[1].strip()
    
    # Based on the data, we can categorize:
    # PR (Partial Responder) and NED2 (remained disease-free for ≥18 months) as positive outcomes (1)
    # PD (Progressive Disease), SD (Stable Disease), and NED1 (remained disease-free for ≤18 months) as less favorable (0)
    if clinical_outcome in ['PR', 'NED2']:
        return 1  # Favorable outcome
    elif clinical_outcome in ['PD', 'SD', 'NED1']:
        return 0  # Less favorable outcome
    else:
        return None  # Unknown or undefined value

def convert_age(value):
    """Convert age values to numeric."""
    # This function is defined but not used since age data is not available
    if not value or ':' not in value:
        return None
    
    age_str = value.split(':', 1)[1].strip()
    try:
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    # This function is defined but not used since gender data is not available
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip().lower()
    if 'female' in gender or 'f' == gender:
        return 0
    elif 'male' in gender or 'm' == gender:
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
# Use the function from the library to save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait_row is not None
if trait_row is not None:
    # Create clinical_data DataFrame using the sample characteristics dictionary from previous output
    # The dictionary structure is {row_index: [list_of_values_for_each_sample]}
    sample_char_dict = {
        0: ['subject id: Patient 10', 'subject id: Patient 11', 'subject id: Patient 12', 'subject id: Patient 13', 'subject id: Patient 14', 'subject id: Patient 15', 'subject id: Patient 16', 'subject id: Patient 17', 'subject id: Patient 18', 'subject id: Patient 19', 'subject id: Patient 2', 'subject id: Patient 20', 'subject id: Patient 21', 'subject id: Patient 22', 'subject id: Patient 23', 'subject id: Patient 24', 'subject id: Patient 25', 'subject id: Patient 26', 'subject id: Patient 27', 'subject id: Patient 28', 'subject id: Patient 29', 'subject id: Patient 3', 'subject id: Patient 30', 'subject id: Patient 31', 'subject id: Patient 32', 'subject id: Patient 33', 'subject id: Patient 34', 'subject id: Patient 35', 'subject id: Patient 4', 'subject id: Patient 5'],
        1: ['patient diagnosis: melanoma'] * 30,  # Same value for all samples
        2: ['cell type: Dendritic Cell'] * 30,    # Same value for all samples
        3: ['dc subtype: day 5 iDC', 'dc subtype: day 6 mDC', 'dc subtype: AdVTMM2/DC'] * 10,  # Repeating pattern
        4: ['clinical outcome: PR', 'clinical outcome: PD', 'clinical outcome: SD', 'clinical outcome: NED1', 'clinical outcome: NED2'] * 6,  # Repeating pattern
        5: ['overall survival (months): 44.33', 'overall survival (months): 14.833', 'overall survival (months): 41.73', 'overall survival (months): 42.53', 'overall survival (months): 28.43', 'overall survival (months): 42.7', 'overall survival (months): 38.6', 'overall survival (months): 4.63', 'overall survival (months): 39.9', 'overall survival (months): 3.5', 'overall survival (months): 14.03', 'overall survival (months): 40.17', 'overall survival (months): 20.27', 'overall survival (months): 3.03', 'overall survival (months): 11.2', 'overall survival (months): 3.27', 'overall survival (months): 0.67', 'overall survival (months): 36.13', 'overall survival (months): 35.87', 'overall survival (months): 11.8', 'overall survival (months): 32.33', 'overall survival (months): 13.63', 'overall survival (months): 37.5', 'overall survival (months): 37.3', 'overall survival (months): 26.57', 'overall survival (months): 1.67', 'overall survival (months): 8.27', 'overall survival (months): 24.27', 'overall survival (months): 1.07', 'overall survival (months): 39.8']
    }
    
    # Create DataFrame from the sample characteristics dictionary
    clinical_data = pd.DataFrame(sample_char_dict)
    
    # Debug: Print structure of clinical_data
    print("Clinical data structure:")
    print(clinical_data.head(2))
    
    # Extract clinical features using geo_select_clinical_features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Debug: Check the structure of selected_clinical_df
    print("\nSelected clinical features structure:")
    print(selected_clinical_df.head(2))
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("\nPreview of extracted clinical features:")
    for key, values in preview.items():
        print(f"{key}: {values}")
    
    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"\nClinical data saved to {out_clinical_data_file}")


Clinical data structure:
                        0                            1  \
0  subject id: Patient 10  patient diagnosis: melanoma   
1  subject id: Patient 11  patient diagnosis: melanoma   

                           2                      3                     4  \
0  cell type: Dendritic Cell  dc subtype: day 5 iDC  clinical outcome: PR   
1  cell type: Dendritic Cell  dc subtype: day 6 mDC  clinical outcome: PD   

                                   5  
0   overall survival (months): 44.33  
1  overall survival (months): 14.833  

Selected clinical features structure:
           0   1   2   3    4   5
Melanoma NaN NaN NaN NaN  1.0 NaN

Preview of extracted clinical features:
0: [nan]
1: [nan]
2: [nan]
3: [nan]
4: [1.0]
5: [nan]

Clinical data saved to ../../output/preprocess/Melanoma/clinical_data/GSE157738.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 53617
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# The identifiers (e.g. '16650001', '16650003', etc.) appear to be numeric values
# These are likely probe IDs from a microarray platform and not standard human gene symbols
# Human gene symbols typically follow nomenclature rules (e.g., BRCA1, TP53)
# These numeric IDs will need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene annotation data to identify which column contains gene symbols
# Based on the preview, we need to determine which column might contain gene symbols
print("Looking for gene symbols in annotation columns:")
symbol_candidates = []

# Check each column in the annotation data for potential gene symbols
for col_name in gene_annotation.columns:
    # Get first 5 non-null values for each column
    sample_values = gene_annotation[col_name].dropna().head(5).tolist()
    print(f"Column '{col_name}' sample values: {sample_values}")
    
    # Look for patterns that might indicate gene symbols 
    if any(extract_human_gene_symbols(str(val)) for val in sample_values):
        symbol_candidates.append(col_name)
        print(f"  - Column '{col_name}' may contain gene symbols")

# If we found no obvious gene symbol column, we need to look deeper
if not symbol_candidates:
    print("\nNo obvious gene symbol column found. Looking at more rows:")
    # Look at the first 10 rows to check for any symbols in other columns
    for col_name in gene_annotation.columns:
        symbols_found = []
        for val in gene_annotation[col_name].dropna().head(50):
            symbols = extract_human_gene_symbols(str(val))
            if symbols:
                symbols_found.extend(symbols)
        if symbols_found:
            print(f"Found potential gene symbols in '{col_name}': {symbols_found[:10]}")
            symbol_candidates.append(col_name)

# If still no symbols found, see if we can check the full content of a column
if not symbol_candidates:
    print("\nChecking full content of specific columns that might contain gene IDs:")
    for col_name in ['GB_ACC', 'SPOT_ID']:
        print(f"\nUnique values in '{col_name}':")
        print(gene_annotation[col_name].dropna().unique()[:20])

# 2. Determine which columns to use for mapping
# 'ID' clearly corresponds to the probe IDs in expression data
id_col = 'ID'

# We need to decide which column contains gene symbols
# If we've identified a column, use it; otherwise use a reasonable default
if symbol_candidates:
    gene_col = symbol_candidates[0]
else:
    # If no clear gene symbol column is found, try using SPOT_ID or GB_ACC
    # SPOT_ID often contains location information that can be mapped to genes
    gene_col = 'GB_ACC'  # Default to GB_ACC as it might contain gene references

print(f"\nUsing '{id_col}' for probe IDs and '{gene_col}' for gene identifiers")

# 3. Create a mapping between probe IDs and gene identifiers
mapping_df = get_gene_mapping(gene_annotation, id_col, gene_col)
print("\nMapping DataFrame preview:")
print(preview_df(mapping_df))

# 4. Apply the mapping to convert probe level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print("\nGene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# 5. Normalize gene symbols (convert to standard form)
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalization:")
print(f"Number of genes after normalization: {len(gene_data)}")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# 6. Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")

Looking for gene symbols in annotation columns:
Column 'ID' sample values: ['16657436', '16657440', '16657445', '16657447', '16657450']


Column 'RANGE_STRAND' sample values: ['+', '+', '+', '+', '+']
Column 'RANGE_START' sample values: [12190.0, 29554.0, 69091.0, 160446.0, 317811.0]
Column 'RANGE_END' sample values: [13639.0, 31109.0, 70008.0, 161525.0, 328581.0]
Column 'total_probes' sample values: [25.0, 28.0, 8.0, 13.0, 36.0]
Column 'GB_ACC' sample values: ['NR_046018', 'NR_024368', 'NR_029406', 'XR_132471', 'NR_047526']
Column 'SPOT_ID' sample values: ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581']


Column 'RANGE_GB' sample values: ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']

No obvious gene symbol column found. Looking at more rows:



Checking full content of specific columns that might contain gene IDs:

Unique values in 'GB_ACC':
['NR_046018' 'NR_024368' 'NR_029406' 'XR_132471' 'NR_047526' 'NM_152486'
 'NM_198317' 'NM_005101' 'NM_198576' 'NR_038869' 'NR_029639' 'NR_029834'
 'NR_029957' 'NM_153339' 'NM_001029885' 'NR_015434' 'NM_001146685'
 'NM_022834' 'NM_080875' 'NM_006983']

Unique values in 'SPOT_ID':
['chr1:12190-13639' 'chr1:29554-31109' 'chr1:69091-70008'
 'chr1:160446-161525' 'chr1:317811-328581' 'chr1:329790-342507'
 'chr1:367640-368634' 'chr1:459656-461954' 'chr1:523009-532878'
 'chr1:557143-566063' 'chr1:696291-697369' 'chr1:714162-745440'
 'chr1:721320-722513' 'chr1:752751-755214' 'chr1:756056-758975'
 'chr1:762988-789791' 'chr1:840487-841186' 'chr1:846815-850735'
 'chr1:860260-879961' 'chr1:895967-901099']

Using 'ID' for probe IDs and 'GB_ACC' for gene identifiers



Mapping DataFrame preview:
{'ID': ['16657436', '16657450', '16657476', '16657485', '16657514'], 'Gene': ['NR_046018', 'NR_024368', 'NR_029406', 'XR_132471', 'NR_047526']}

Gene expression data after mapping:
Number of genes: 0
First few gene symbols:
[]

Gene expression data after normalization:
Number of genes after normalization: 0
First few normalized gene symbols:
[]

Gene expression data saved to ../../output/preprocess/Melanoma/gene_data/GSE157738.csv
